This is a section sequencing classifier. We used LSTM for section sequencing.

https://stackoverflow.com/questions/38189070/how-do-i-create-a-variable-length-input-lstm-in-keras

https://machinelearningmastery.com/sequence-prediction/

https://stackoverflow.com/questions/43117654/many-to-many-sequence-prediction-with-different-sequence-length

In [ ]:
import sklearn.preprocessing as skprep

import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import datasets
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import preprocessing
from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras import callbacks
from tensorflow.contrib.keras import utils

from sklearn import preprocessing

from collections import defaultdict
import pandas as pd
import numpy as np
import os
import json
import pickle
from datetime import datetime
import sys
from sklearn import metrics,cross_validation
from tensorflow.contrib.keras.python.keras.layers.wrappers import Bidirectional

tf.logging.set_verbosity(tf.logging.WARN)


In [ ]:
# encdoing labels into numbers
class_list={}
for x in open("data/class_list.txt","r").readlines():
    x=x.strip()
    class_list[x.split(":")[0]]=x.split(":")[1].lower()

lencoder = preprocessing.LabelEncoder()
lencoder.fit_transform(class_list.values())


In [ ]:
# Process training and test data
X_train={}
X_train["Sequence"]=[]
train_f=open("data/train_section_sequencing_classifier","r")

for line in train_f:
    line=list(lencoder.transform(line.strip().split(",")))
    if len(line)<1:
        continue
    X_train["Sequence"].append(line)



X_test={}
X_test["Sequence"]=[]
test_f=open("data/test_section_sequencing_classifier","r")

for line in test_f:
    line=list(lencoder.transform(line.strip().split(",")))
    if len(line)<1:
        continue

    X_test["Sequence"].append(line)



In [ ]:
print len(X_train["Sequence"])

In [ ]:
max_features=16
num_mem_units = 64
size_batch = 1
num_timesteps = 1
num_features = 1
num_targets = 1
num_epochs = 100


In [ ]:
model = models.Sequential()
model.add(layers.LSTM(num_mem_units, stateful=True,  batch_input_shape=(size_batch, num_timesteps, num_features),
  return_sequences=True))
model.add(layers.Dense(num_targets, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

print model.summary()

In [ ]:
range_act = (0, 1) # sigmoid
range_features = np.array([0, max_features]).reshape(-1, 1)
normalizer = skprep.MinMaxScaler(feature_range=range_act)
normalizer.fit(range_features)

In [ ]:
reset_state = callbacks.LambdaCallback(on_epoch_end=lambda *_ : model.reset_states())


In [ ]:
# training
for seq in X_train['Sequence']:
    X = seq[:-1]
    y = seq[1:] # predict next element
    X_norm = normalizer.transform(np.array(X).reshape(-1, 1)).reshape(-1, num_timesteps, num_features)
    y_norm = normalizer.transform(np.array(y).reshape(-1, 1)).reshape(-1, num_timesteps, num_targets)
    model.fit(X_norm, y_norm, epochs=num_epochs, batch_size=size_batch, shuffle=False,verbose=1,
      callbacks=[reset_state])

In [ ]:
# prediction
index=0
for seq in X_test['Sequence']:
    model.reset_states() 
    for istep in range(len(seq)-1): # input up to not incl last
        val = seq[istep]
        X = np.array([val]).reshape(-1, 1)
        X_norm = normalizer.transform(X).reshape(-1, num_timesteps, num_features)
        y_norm = model.predict(X_norm)
    yhat = int(normalizer.inverse_transform(y_norm[0])[0, 0])
    y = seq[-1] # last
    put = '{0} predicts {1:d}, expecting {2:d}'.format(', '.join(str(val) for val in seq[:-1]), yhat, y)
    print(put)

    if index>10:
        break
    index+=1
    